<a href="https://colab.research.google.com/github/georgesteve/Research-Assitant-ML-ROJ/blob/main/Semana_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#[Forecasting Global Ionospheric TEC Using Deep Learning Approach](https://doi.org/10.1029/2020SW002501)

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [ ]:
ls

 0001.jpg
 85192060-Los-cincuenta-anos-de-Jicamarca.pdf
'98 FORMATO  DEDICATORIA APG.doc (1).docx'
'ATRAD presentación 1 ROJ.gslides'
 Classroom/
'Colab Notebooks'/
'COMPENDIO FAJARDO SORIA'/
'Copia de Checklist para discusiones públicas.gsheet'
'Copia de CitiesExt.csv.gsheet'
'Copia de Copia de Copia de TESIS FAJARDO SORIA GEORGE Presentación Final.gslides'
'Copia de deltaV station keeping .gdoc'
'Copia de Manual Sesiones de Prueba de Amanecidas Hackspace.gdoc'
'Copia de PlayersExt.csv.gsheet'
'Copia de Process Flow Diagram Template.gdraw'
'Copia de TESIS FAJARDO SORIA GEORGE Presentación Final (1).gslides'
'Copia de TESIS FAJARDO SORIA GEORGE Presentación Final.gslides'
'Copia de yqw-xmbf-pkw - 3 de dic. de 2020.gjam'
'coreupgrade retos'/
 Cuentas.gsheet
 curso-frances.pdf
'Documento sin título (1).gdoc'
'Documento sin título (2).gdoc'
'Documento sin título.gdoc'
'ECI TESIS FAJARDO SORIA GEORGE Presentación Final (1).gslides'
'ECI TESIS FAJARDO SORIA GEORGE Presentación Fin

Este paper utiliza RNN con arquitecturas LSTM para predecir condiciones de TEC.


In [ ]:
https://github.com/georgesteve/Research-Assitant-ML-ROJ.git

In [34]:
import pandas
URL = "https://raw.githubusercontent.com/georgesteve/Research-Assitant-ML-ROJ/main/gfajardo_uNz2xU-2020-2.csv"
Data = pandas.read_csv(URL,index_col=False) 
df=Data.drop(columns=['Unnamed: 0'])


In [35]:
print(df)

        Año  Day  Tiempo  PRN    S4     Az   Elv
0        20  306      22    3  0.10  300.9  24.6
1        20  306      22    8  0.05  199.5  47.2
2        20  306      22   10  0.40  147.1  14.3
3        20  306      22   11  0.08  210.6  26.2
4        20  306      22   16  0.04  342.4  42.3
...     ...  ...     ...  ...   ...    ...   ...
452982   20  337   85540   21  0.05  159.0  48.7
452983   20  337   85540   22  0.08  206.3  42.2
452984   20  337   85540   27  0.06   22.7  36.8
452985   20  337   85540   31  0.20   89.3  15.1
452986   20  337   85540  138  0.04  288.3  50.6

[452987 rows x 7 columns]


In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

def trend(time, slope=0):
    return slope * time

def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, 0.1)  
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Create the series
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)
# Update with noise
series += noise(time, noise_level, seed=42)

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000